In [11]:
let rec of_list cs = 
  match cs with
  | [] -> ""
  | c::cs -> String.make 1 c ^ of_list cs;;


val of_list : char list -> string = <fun>


In [3]:
type regexp = Zero | One | Char of char | Cat of regexp * regexp | Sum of regexp * regexp | Star of regexp;;

let rec of_regexp_1 r = 
  match r with
  | Zero -> "Zero"
  | One -> "One"
  | Char c -> Printf.sprintf "Char (%c)" c
  | Cat (r1, r2) -> Printf.sprintf "Cat (%s, %s)" (of_regexp_1 r1) (of_regexp_1 r2)
  | Sum (r1, r2) -> Printf.sprintf "Sum (%s, %s)" (of_regexp_1 r1) (of_regexp_1 r2)
  | Star (r1) -> Printf.sprintf "Star (%s)" (of_regexp_1 r1);;

let rec accept r s k = 
  match r with
  | Zero -> false
  | One -> k s
  | Char c -> 
    (match s with
    | c2 :: s2 -> (c = c2) && (k s2)
    | [] -> false)
  | Cat (r1, r2) -> accept r1 s (fun s2 -> accept r2 s2 k)
  | Sum (r1, r2) -> 
    accept r1 s k || accept r2 s k
  | Star r -> 
    let rec accept_star r s k = 
        k s || accept r s (fun s2 -> 
          print_endline (of_list s2);
          print_endline (of_list s);
          s != s2 && accept_star r s2 k)
    in accept_star r s k;;

let m r s = accept r s (fun s -> s = []);;

m (Star (Star (Char '1'))) ['1'; '1'; '0']


type regexp =
    Zero
  | One
  | Char of char
  | Cat of regexp * regexp
  | Sum of regexp * regexp
  | Star of regexp


val of_regexp_1 : regexp -> string = <fun>


val accept : regexp -> char list -> (char list -> bool) -> bool = <fun>


val m : regexp -> char list -> bool = <fun>


110
110
10
110
10
110
10
10
0
10
0
10
0
0
0
10
0
110
0
0


- : bool = false


In [4]:

let rec accept r k = 
  match r with
  | Zero -> fun _ -> false
  | One -> fun s -> k s
  | Char c -> 
    (fun s -> 
    (match s with
    | c2 :: s2 -> (c = c2) && (k s2)
    | [] -> false))
  | Cat (r1, r2) -> 
    let partial_2 = accept r2 k in 
    let lifted_partial_2 = partial_2 in
    let partial_1 = accept r1 (fun s2 -> lifted_partial_2 s2)
    in
    (fun s -> 
      partial_1 s)
  | Sum (r1, r2) -> 
    let partial_1 = accept r1 k in
    let partial_2 = accept r2 k in
    fun s -> 
      partial_1 s || partial_2 s
  | Star r -> 
    print_endline (Printf.sprintf "Recursive call on %s" (of_regexp_1 (Star (r))));
    let new_loop = (fun partial_star s1 s2 -> 
      print_endline (Printf.sprintf "Star call on %s %s" (of_list (s1)) (of_list s2));
      s1 != s2 && partial_star s2
    ) in
    let rec accept_star r k = 
      fun s -> 
        print_endline (Printf.sprintf "Accept star call on %s" (of_regexp_1 r));
        k s || accept r (new_loop (accept_star r k) s) s
    in
    let acc_star_partial = accept_star r k in
    fun s -> 
      acc_star_partial s;;

let m r s = accept r (fun s -> s = []) s;;

m (Star (Star (Char '1'))) ['1'; '0']

val accept : regexp -> (char list -> bool) -> char list -> bool = <fun>


val m : regexp -> char list -> bool = <fun>


Recursive call on Star (Star (Char (1)))
Accept star call on Star (Char (1))
Recursive call on Star (Char (1))
Accept star call on Char (1)
Star call on 10 10
Star call on 10 0
Accept star call on Char (1)
Star call on 10 0
Accept star call on Star (Char (1))
Recursive call on Star (Char (1))
Accept star call on Char (1)
Star call on 0 0


- : bool = false


In [7]:

let rec accept r k = 
  match r with
  | Zero -> fun _ -> false
  | One -> fun s -> k s
  | Char c -> 
    (fun s -> 
    if (match s with
    | c2 :: s2 -> ((c = c2) && (k s2)) 
    | [] -> false) then (print_endline "true"; true) else (print_endline "false"; false))
  | Cat (r1, r2) -> 
    let partial_2 = accept r2 k in 
    let lifted_partial_2 = partial_2 in
    let partial_1 = accept r1 (fun s2 -> lifted_partial_2 s2)
    in
    (fun s -> 
      partial_1 s)
  | Sum (r1, r2) -> 
    let partial_1 = accept r1 k in
    let partial_2 = accept r2 k in
    fun s -> 
      partial_1 s || partial_2 s
  | Star r -> 
    print_endline (Printf.sprintf "Recursive call on %s" (of_regexp_1 (Star (r))));
    let new_loop = (fun partial_star s1 s2 -> 
      print_endline (Printf.sprintf "Star call at %s on %s %s" (of_regexp_1 (Star r)) (of_list (s1)) (of_list s2));
      s1 != s2 && partial_star s2
    ) in
    let rec accept_star r k = 
      fun s -> 
        print_endline (Printf.sprintf "Accept star call at %s on %s" (of_regexp_1 r) (of_list s));
        k s || accept r (new_loop (accept_star r k) s) s
    in
    let acc_star_partial = accept_star r k in
    fun s -> 
      acc_star_partial s;;

let m r s = accept r (fun s -> s = []) s;;

m (Star (Star (Char '1'))) ['1'; '0']

val accept : regexp -> (char list -> bool) -> char list -> bool = <fun>


val m : regexp -> char list -> bool = <fun>


Recursive call on Star (Star (Char (1)))
Accept star call at Star (Char (1)) on 10
Recursive call on Star (Char (1))
Accept star call at Char (1) on 10
Star call at Star (Star (Char (1))) on 10 10
Star call at Star (Char (1)) on 10 0
Accept star call at Char (1) on 0
Star call at Star (Star (Char (1))) on 10 0
Accept star call at Star (Char (1)) on 0
Recursive call on Star (Char (1))
Accept star call at Char (1) on 0
Star call at Star (Star (Char (1))) on 0 0
false
false
false


- : bool = false


In [8]:
m (Star (Star (Char '1'))) ['1'; '0']

Recursive call on Star (Star (Char (1)))
Accept star call at Star (Char (1)) on 10
Recursive call on Star (Char (1))
Accept star call at Char (1) on 10
Star call at Star (Star (Char (1))) on 10 10
Star call at Star (Char (1)) on 10 0
Accept star call at Char (1) on 0
Star call at Star (Star (Char (1))) on 10 0
Accept star call at Star (Char (1)) on 0
Recursive call on Star (Char (1))
Accept star call at Char (1) on 0
Star call at Star (Star (Char (1))) on 0 0
false
false
false


- : bool = false


In [10]:
type regexp2 =
Empty
| Plus of (regexp2 * regexp2)
| Times of (regexp2 * regexp2)
| Star of regexp2
| Const of char
;;

let rec of_regexp_2 r = 
  match r with 
  | Plus (r1, r2) -> Printf.sprintf "Plus (%s, %s)" (of_regexp_2 r1) (of_regexp_2 r2)
  | Times (r1, r2) -> Printf.sprintf "Times (%s, %s)" (of_regexp_2 r1) (of_regexp_2 r2)
  | Star (r1) -> Printf.sprintf "Star (%s)" (of_regexp_2 r1)
  | Const c -> Printf.sprintf "Const (%c)" c
  | Empty -> "Empty"


type regexp2 =
    Empty
  | Plus of (regexp2 * regexp2)
  | Times of (regexp2 * regexp2)
  | Star of regexp2
  | Const of char


val of_regexp_2 : regexp2 -> string = <fun>


In [19]:
let rec acc2 = 
  fun (exp) ->    
      match exp with
      | Empty -> 
          (fun (cont) -> ( 
              cont
          ))
      | Plus (e1, e2) -> 
          let res1 = acc2 e1 in
          let res2 = acc2 e2 in
          (fun (cont) ->
              let unboxed_res1 = (res1 (cont)) in
              let unboxed_res2 = (res2 (cont)) in
              fun (str2, str, loop) -> 
                  (*Short circuiting OR*)
                  (unboxed_res1 (str2, str, loop)) || (unboxed_res2 (str2, str, loop)))
      | Times (e1, e2) ->
          let res1_ = acc2 e1 in
          let res2_ = acc2 e2 in
          (fun (cont) ->
              res1_ (res2_ (cont))
          ) 
      | Const (c) ->
          let lifted_unboxed_c = c in
          (fun (cont) -> ( 
              let unboxed_cont = cont in
              fun (str2, str, loop) -> (
                  match str with
                  | [] -> false
                  | x::xs -> 
                      (*Shortcircuiting AND*)
                      (x = (lifted_unboxed_c)) && (unboxed_cont (str2, xs, loop))
              )
          ))
      | Star (e) -> 
            print_endline (Printf.sprintf "Recursive call on %s" (of_regexp_2 (Star (e))));
          let res = acc2 e in 
          (fun (cont) -> 
              let unboxed_cont = cont in
              (fun (str2, str, loop) -> 
                  (*Construct cont*)
                  (* let lifted_unboxed_old_str = str in *)
                  let new_cont = (fun (str2, str, loop) ->  
                    print_endline (Printf.sprintf "Star call at %s on %s %s" (of_regexp_2 (Star (e))) (of_list (str2)) (of_list str));
                    str2 != str && loop str
                  )
                  in
                  let unboxed_acc2_e_new_cont = res (new_cont) in
                  let rec star_loop = 
                      fun (s) ->
                          (*If we don't have anything left we just call the continuation*)
                              (*Shortcircuiting OR*)
                              print_endline (Printf.sprintf "Accept star call at %s on %s" (of_regexp_2 (e)) (of_list s));
                            (unboxed_cont (str2, s, loop)) || unboxed_acc2_e_new_cont (s, s, star_loop)
                  in
                  star_loop str
              )
          )
  ;;

let accept r x = acc2 r (fun (_, s, _) -> s = []) ([], x, fun _ -> false)

val acc2 :
  regexp2 ->
  (char list * char list * (char list -> bool) -> bool) ->
  char list * char list * (char list -> bool) -> bool = <fun>


val accept : regexp2 -> char list -> bool = <fun>


In [26]:
accept (Star (Star (Star (Const '1')))) ['0'; '1'; '1'; '0']

Recursive call on Star (Star (Star (Const (1))))
Recursive call on Star (Star (Const (1)))
Recursive call on Star (Const (1))
Accept star call at Star (Star (Const (1))) on 0110
Accept star call at Star (Const (1)) on 0110
Star call at Star (Star (Star (Const (1)))) on 0110 0110
Accept star call at Const (1) on 0110
Star call at Star (Star (Const (1))) on 0110 0110


- : bool = false
